# Ungraded Lab: TFDV Exercise


# Ungraded Lab: TFDV Exercise


TFDV fournit des informations sur certaines questions clés du processus d'analyse de données telles que :

* Quelles sont les statistiques sous-jacentes de mes données ?

* Comment mon jeu de données d'entraînement est-il organisé ?

* Comment mon jeu de données d'évaluation et de mise en service se compare-t-il au jeu de données d'entraînement ?

* Comment puis-je trouver et corriger les anomalies de données ?

La figure ci-dessous résume le processus habituel de TFDV :

<img src='img/tfdv.png' alt='picture of tfdv workflow'>

En MLOps (ops signifiant "opérations"), les termes "feature skew", "schema skew" et "distribution skew" font référence aux différents types de déséquilibres qui peuvent survenir dans les données d'un modèle de machine learning au cours de son cycle de vie.

Voici ce que ces termes signifient en détail :

**Feature skew** : cela se produit lorsqu'une ou plusieurs colonnes (ou features) d'un jeu de données ont des valeurs manquantes, aberrantes ou non valides. Cela peut affecter négativement la qualité du modèle et sa performance.

**Schema skew** : cela se produit lorsqu'il y a des différences dans la structure ou le type des données entre les différentes versions d'un jeu de données. Par exemple, si une colonne a été ajoutée ou supprimée d'une version à l'autre, ou si le type de données d'une colonne a changé. Cela peut entraîner des erreurs lors de l'exécution du modèle.

**Distribution skew** : cela se produit lorsqu'il y a des différences dans la distribution des données entre les différentes versions d'un jeu de données. Par exemple, si la proportion de valeurs positives et négatives dans une colonne a changé d'une version à l'autre. Cela peut affecter la qualité du modèle et sa performance.

Il est important de surveiller et de corriger ces déséquilibres dans les données pour assurer la qualité et la robustesse du modèle de machine learning.
<br/>
Comme indiqué, nous pouvons utiliser TFDV pour calculer des statistiques descriptives des données d'entraînement et générer un schéma. Nous pouvons alors valider de nouveaux jeux de données (par exemple, le jeu de données de service de nos clients) contre ce schéma pour détecter et corriger les anomalies. Cela permet d'éviter les différents types de distorsion. De cette façon, nous pouvons être sûrs que notre modèle s'entraîne sur des données ou prédit des données qui sont cohérentes avec les types de caractéristiques et la distribution attendus.
- **Générer et visualiser des statistiques à partir d'un jeu de données**
- **Détecter et corriger les anomalies dans un jeu de données d'évaluation**



## Package Installation and Imports

In [1]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
import pandas as pd

from sklearn.model_selection import train_test_split
from util import add_extra_rows

from tensorflow_metadata.proto.v0 import schema_pb2

print('TFDV Version: {}'.format(tfdv.__version__))
print('Tensorflow Version: {}'.format(tf.__version__))

TFDV Version: 1.3.0
Tensorflow Version: 2.6.0


## Télécharger le jeu de données

Vous allez travailler avec le [Census Income Dataset] (http://archive.ics.uci.edu/ml/datasets/Census+Income), un ensemble de données qui peut être utilisé pour prédire si un individu gagne plus ou moins de 50 000 dollars américains par an. Le résumé des noms d'attributs avec les descriptions/valeurs attendues est présenté ci-dessous et vous pouvez en savoir plus [dans ce fichier de description des données](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names).


* **âge** : continu.
**classe de travail** : Privé, Indépendant-non-inc, Indépendant-inc, Fédéral-gouv, Local-gouv, Etat-gouv, Sans salaire, Jamais travaillé.
**fnlwgt** : continu.
** **éducation** : Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
** **numéro d'éducation** : continu.
**Statut matrimonial** : Marié(e)-conjoint(e), Divorcé(e), Jamais marié(e), Séparé(e), Veuf(ve), Conjoint(e)-absent(e), Conjoint(e)-AF.
**Profession** : Support technique, Artisanat-réparation, Autres services, Ventes, Cadres supérieurs, Prof-spécialité, Manutentionnaires-nettoyeurs, Machines-op-inspct, Administration-classe, Agriculture-pêche, Transport-mouvement, Services domestiques privés, Services de protection, Forces armées.
* **relation** : Femme, Enfant à charge, Mari, Pas dans la famille, Autre parent, Célibataire.
**Race** : Blanc, Asiatique-Pac-Islander, Amérindien-Esquimau, Autre, Noir.
**sexe** : Femme, Homme.
**Gain de capital** : continu.
* **perte de capital** : continu.
* **heures-per-week** : continu.
**Pays d'origine** : Etats-Unis, Cambodge, Angleterre, Puerto-Rico, Canada, Allemagne, Etats-Unis d'Amérique (Guam-USVI-etc), Inde, Japon, Grèce, Sud, Chine, Cuba, Iran, Honduras, Philippines, Italie, Pologne, Jamaïque, Vietnam, Mexique, Portugal, Irlande, France, République dominicaine, Laos, Equateur, Taiwan, Haïti, Colombie, Hongrie, Guatemala, Nicaragua, Ecosse, Thaïlande, Yougoslavie, El-Salvador, Trinadad&Tobago, Pérou, Hong, Hollande.

Chargeons l'ensemble de données et divisons-le en ensembles de formation et d'évaluation. Nous ne les mélangerons pas pour obtenir des résultats cohérents dans ce notebook de démonstration, mais vous devriez le faire dans des projets réels.


In [2]:
# Read in the training and evaluation datasets
df = pd.read_csv('data/adult.data', skipinitialspace=True)

# Split the dataset. Do not shuffle for this demo notebook.
train_df, eval_df = train_test_split(df, test_size=0.2, shuffle=False)

Let's see the first few columns of the train and eval sets.

In [3]:
# Preview the train set
train_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
# Preview the eval set
eval_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
26048,30,Private,270886,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,40,United-States,<=50K
26049,21,Private,216129,HS-grad,9,Never-married,Other-service,Own-child,White,Male,0,0,35,United-States,<=50K
26050,33,Private,189368,Some-college,10,Married-civ-spouse,Transport-moving,Husband,Black,Male,0,0,40,United-States,>50K
26051,19,?,141418,Some-college,10,Never-married,?,Own-child,White,Male,0,0,15,United-States,<=50K
26052,19,Private,306225,HS-grad,9,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,25,United-States,<=50K


À partir de ces quelques colonnes, vous pouvez avoir une première impression des données. Vous remarquerez que la plupart sont des chaînes de caractères et des nombres entiers. Il y a aussi des colonnes qui sont principalement des zéros.

### Ajout de lignes supplémentaires

Pour démontrer comment TFDV peut détecter des anomalies plus tard, vous allez ajouter quelques lignes supplémentaires au jeu de données d'évaluation. Celles-ci sont soit mal formées, soit ont des valeurs qui déclencheront certaines alarmes plus tard dans ce cahier. Le code pour les ajouter peut être vu dans la fonction `add_extra_rows()` de `util.py` trouvé dans votre espace de travail Jupyter. Vous pourrez y jeter un coup d'oeil plus tard et même le modifier une fois que vous aurez terminé l'exercice. Pour l'instant, exécutons simplement la fonction et ajoutons les rangées que nous avons définies par défaut .

In [5]:
# add extra rows
eval_df = add_extra_rows(eval_df)

# preview the added rows
eval_df.tail(4)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
6513,46,NaN,257473,Bachelors,8,Married-civ-spouse,Plumber,Husband,Other,Male,1000,0,41,Australia,>50K
6514,0,Private,257473,Masters,8,Married-civ-spouse,Adm-clerical,Wife,Asian,Female,0,0,40,Pakistan,>50K
6515,1000,Private,257473,Masters,8,Married-civ-spouse,Prof-specialty,Husband,Black,Male,0,0,20,Cameroon,<=50K
6516,25,?,257473,Masters,8,Married-civ-spouse,gamer,Husband,Asian,Female,0,0,50,Mongolia,<=50K


## Generate and visualize training dataset statistics

Vous pouvez maintenant calculer et visualiser les statistiques de votre ensemble de données d'entraînement. TFDV accepte trois formats d'entrée : TFRecord de TensorFlow, Dataframe Pandas et fichier CSV. Dans cet exercice, vous allez introduire les Dataframes Pandas que vous avez générés à partir de la division train-test. 

Vous pouvez calculer les statistiques de votre ensemble de données en utilisant la méthode [`generate_statistics_from_dataframe()`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/generate_statistics_from_dataframe). Sous le capot, l'analyse est distribuée via [Apache Beam](https://beam.apache.org/), ce qui lui permet de s'adapter à de grands ensembles de données.

Les résultats renvoyés par cette étape pour les données numériques et catégorielles sont résumés dans ce tableau :

| Données numériques Données catégorielles
|:-:|:-:|
|Nombre d'enregistrements de données Nombre d'enregistrements de données
|Pourcentage d'enregistrements manquants. Pourcentage d'enregistrements manquants.
|Moyenne, std, min, max|enregistrements uniques|.
|% de valeurs nulles|Longueur moyenne des chaînes de caractères|.


In [6]:
# Generate training dataset statistics
train_stats = tfdv.generate_statistics_from_dataframe(train_df)

Une fois que vous avez généré les statistiques, vous pouvez facilement visualiser vos résultats avec la méthode [`visualize_statistics()`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/visualize_statistics). Cette méthode affiche une [interface Facets](https://pair-code.github.io/facets/) et est très utile pour repérer si vous avez un grand nombre de données manquantes ou un écart type élevé. Exécutez la cellule ci-dessous et explorez les différents paramètres de l'interface de sortie (par exemple, trier par, ordre inverse, recherche de caractéristiques).

In [8]:
# Visualize training dataset statistics
tfdv.visualize_statistics(train_stats)

## Infer data schema

L'étape suivante consiste à créer un schéma de données pour décrire votre ensemble de trains. En termes simples, un schéma décrit les caractéristiques standard de vos données, telles que les types de données des colonnes et la plage de valeurs des données attendues. Le schéma est créé sur un ensemble de données que vous considérez comme une référence, et peut être réutilisé pour valider d'autres ensembles de données entrants.

Avec les statistiques calculées, TFDV vous permet de générer automatiquement une version initiale du schéma en utilisant la méthode [`infer_schema()`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/infer_schema). Cette méthode renvoie un Schema [protocol buffer](https://developers.google.com/protocol-buffers) contenant le résultat. Comme mentionné dans le [TFX paper](http://stevenwhang.com/tfx_paper.pdf) (Section 3.3), les résultats de l'inférence du schéma peuvent être résumés comme suit :

* Le type attendu de chaque élément.
* La présence attendue de chaque caractéristique, en termes de nombre minimum et de fraction d'exemples qui doivent contenir la caractéristique.
* La valence attendue de la caractéristique dans chaque exemple, c'est-à-dire,le nombre minimum et maximum de valeurs.
* Le domaine attendu d'une caractéristique, c'est-à-dire le petit univers de valeurs pour une caractéristique de type chaîne, ou la gamme de valeurs pour une caractéristique de type chaîne de caractères, ou la plage pour une caractéristique de type nombre entier.


In [9]:
# Infer schema from the computed statistics.
schema = tfdv.infer_schema(statistics=train_stats)

# Display the inferred schema
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'workclass',STRING,required,,'workclass'
'fnlwgt',INT,required,,-
'education',STRING,required,,'education'
'education-num',INT,required,,-
'marital-status',STRING,required,,'marital-status'
'occupation',STRING,required,,'occupation'
'relationship',STRING,required,,'relationship'
'race',STRING,required,,'race'


,Values
Domain,
'workclass',"'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"
'education',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'marital-status',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'occupation',"'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'relationship',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'race',"'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
'sex',"'Female', 'Male'"
'native-country',"'?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
'label',"'<=50K', '>50K'"


## Generate and visualize evaluation dataset statistics

Après avoir généré le schéma, l'étape suivante consiste à examiner l'ensemble de données d'évaluation. Vous commencerez par calculer ses statistiques, puis vous les comparerez à celles de la formation. Il est important que les caractéristiques numériques et catégorielles des données d'évaluation appartiennent à peu près à la même gamme que les données d'apprentissage. Sinon, vous risquez d'avoir une distribution biaisée qui affectera négativement la précision de votre modèle.

TFDV vous permet de générer les statistiques des jeux de données d'apprentissage et d'évaluation côte à côte. Vous pouvez utiliser la fonction [`visualize_statistics()`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/visualize_statistics) et passer des paramètres supplémentaires pour superposer les statistiques des deux ensembles de données (référencées comme statistiques de gauche et de droite). Voyons quels sont ces paramètres :

- `lhs_statistics` : Paramètre obligatoire. Attend une instance de `DatasetFeatureStatisticsList`.

- `rhs_statistics` : Attend une instance de `DatasetFeatureStatisticsList` à comparer avec `lhs_statistics`.

- `lhs_name` : Nom de l'ensemble de données `lhs_statistics`.

- `rhs_name` : Nom de l'ensemble de données `rhs_statistics`.

In [10]:
# Generate evaluation dataset statistics
eval_stats = tfdv.generate_statistics_from_dataframe(eval_df)

# Compare training with evaluation
tfdv.visualize_statistics(
    lhs_statistics=eval_stats, 
    rhs_statistics=train_stats, 
    lhs_name='EVAL_DATASET', 
    rhs_name='TRAIN_DATASET'
)

Nous vous encourageons à observer les résultats générés et à basculer les menus pour vous entraîner à manipuler la visualisation (par exemple, trier par manquants/zéros). Vous remarquerez que TFDV détecte les lignes mal formées que nous avons introduites plus tôt. Tout d'abord, les valeurs `min` et `max` de la ligne `age` indiquent respectivement `0` et `1000`. Nous savons que ces valeurs n'ont pas de sens si l'on parle d'adultes actifs. Deuxièmement, la ligne "classe de travail" dans les caractéristiques catégorielles indique que "0,02%" des données manquent cet attribut particulier. Supprimons ces lignes pour rendre les données plus propres.

In [11]:
# filter the age range
eval_df = eval_df[eval_df['age'] > 16]
eval_df = eval_df[eval_df['age'] < 91]

# drop missing values
eval_df.dropna(inplace=True)

Vous pouvez alors calculer à nouveau les statistiques et voir la différence dans les résultats.

In [12]:
# Generate evaluation dataset statistics
eval_stats = tfdv.generate_statistics_from_dataframe(eval_df)

# Compare training with evaluation
tfdv.visualize_statistics(
    lhs_statistics=eval_stats, 
    rhs_statistics=train_stats, 
    lhs_name='EVAL_DATASET', 
    rhs_name='TRAIN_DATASET'
)

## Calculate and display evaluation anomalies

You can use your reference schema to check for anomalies such as new values for a specific feature in the evaluation data. Detected anomalies can either be considered a real error that needs to be cleaned, or depending on your domain knowledge and the specific case, they can be accepted. 

Let's detect and display evaluation anomalies and see if there are any problems that need to be addressed.

In [13]:
# Check evaluation data for errors by validating the evaluation dataset statistics using the reference schema
anomalies =  tfdv.validate_statistics(statistics=eval_stats, schema=schema)

# Visualize anomalies
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'occupation',Unexpected string values,Examples contain values missing from the schema: gamer (<1%).
'native-country',Unexpected string values,Examples contain values missing from the schema: Mongolia (<1%).
'race',Unexpected string values,Examples contain values missing from the schema: Asian (<1%).


## Révision du schéma

Comme le montrent les résultats ci-dessus, TFDV est capable de détecter les irrégularités restantes que nous avons introduites précédemment. Les descriptions courtes et longues nous indiquent ce qui a été détecté. Comme prévu, il y a des valeurs de chaîne pour `race`, `native-country` et `occupation` qui ne sont pas trouvées dans le domaine du schéma de l'ensemble d'entraînement (vous pourriez voir un résultat différent si le mélange des ensembles de données était appliqué). Ce que vous décidez de faire à propos des anomalies dépend de votre connaissance du domaine des données. Si une anomalie indique une erreur de données, les données sous-jacentes doivent être corrigées. Sinon, vous pouvez mettre à jour le schéma pour inclure les valeurs dans l'ensemble de données d'évaluation.

TFDV fournit un ensemble de méthodes et de paramètres utilitaires que vous pouvez utiliser pour réviser le schéma inféré. Cette [référence] (https://www.tensorflow.org/tfx/data_validation/anomalies) énumère les types d'anomalies et les paramètres que vous pouvez modifier, mais nous ne nous concentrerons ici que sur quelques-uns d'entre eux.

- Vous pouvez assouplir la fraction minimale des valeurs qui doivent provenir du domaine d'une caractéristique particulière (comme décrit par `ENUM_TYPE_UNEXPECTED_STRING_VALUES` dans la [référence](https://www.tensorflow.org/tfx/data_validation/anomalies)) :

```python
tfdv.get_feature(schema, 'feature_column_name').distribution_constraints.min_domain_mass = <float : 0.0 à 1.0>
```

- Vous pouvez ajouter une nouvelle valeur au domaine d'une caractéristique particulière :

```python
tfdv.get_domain(schema, 'feature_column_name').value.append('string')
```

Utilisons-les dans la section suivante.

## Fixer les anomalies dans le schéma

Disons que nous voulons accepter les anomalies de chaîne signalées comme valides. Si vous voulez tolérer une fraction de valeurs manquantes dans l'ensemble de données d'évaluation, vous pouvez le faire comme suit :

In [ ]:
#Relâchez la fraction minimale des valeurs qui doivent provenir du domaine pour la caractéristique `native-country`.
country_feature = tfdv.get_feature(schema, 'native-country')
country_feature.distribution_constraints.min_domain_mass = 0.9

#Relache la fraction minimale des valeurs qui doivent provenir du domaine pour la caractéristique `occupation`.
occupation_feature = tfdv.get_feature(schema, 'occupation')
occupation_feature.distribution_constraints.min_domain_mass = 0.9

Si l'on veut ajouterque des valeurs valides au domaine, on peut procéder comme suit :

In [ ]:
# Add new value to the domain of the feature `race`
race_domain = tfdv.get_domain(schema, 'race')
race_domain.value.append('Asian')

En outre, on peut également restreindre la plage d'une caractéristique numérique. Cela permettra de savoir si une valeur est invalide sans avoir à l'inspecter visuellement (par exemple, les valeurs invalides de `l'âge` plus tôt).

In [ ]:
# Restrict the range of the `age` feature
tfdv.set_domain(schema, 'age', schema_pb2.IntDomain(name='age', min=17, max=90))

# Display the modified schema. Notice the `Domain` column of `age`.
tfdv.display_schema(schema)

With these revisions, running the validation should now show no anomalies.

In [ ]:
# Validate eval stats after updating the schema 
updated_anomalies = tfdv.validate_statistics(eval_stats, schema)
tfdv.display_anomalies(updated_anomalies)

## Examining dataset slices

TFDV  permet également d'analyser des tranches spécifiques de notre jeu de données. Ceci est particulièrement utile si nous voulons vérifier si un type de caractéristique est bien représenté dans l'ensemble de données. Prenons un exemple où nous voulons comparer les statistiques des participants masculins et féminins.

Tout d'abord, on utilise la méthode [`get_feature_value_slicer`](https://github.com/tensorflow/data-validation/blob/master/tensorflow_data_validation/utils/slicing_util.py#L48) du `slicing_util` pour obtenir les caractéristiques que nous voulons examiner. On peut le spécifier en passant un dictionnaire à l'argument `features`. 
 "Si vous voulez obtenir le domaine entier d'une caractéristique, alors vous pouvez associer le nom de la caractéristique avec `None` comme montré ci-dessous. Cela signifie que vous obtiendrez des tranches pour les entrées `Male` et `Female`. Ceci renvoie une fonction qui peut être utilisée pour extraire la tranche de la caractéristique en question."

In [ ]:
from tensorflow_data_validation.utils import slicing_util

slice_fn = slicing_util.get_feature_value_slicer(features={'sex': None})

Avec la fonction slice prête, vous pouvez maintenant générer les statistiques. Vous devez dire à TFDV que vous avez besoin de statistiques pour les caractéristiques que vous avez définies et vous pouvez le faire à travers l'argument `slice_functions` de [`tfdv.StatsOptions`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/StatsOptions). Préparons cela dans la cellule ci-dessous. Notez que vous devez aussi passer le schéma.

In [ ]:
# Declare stats options
slice_stats_options = tfdv.StatsOptions(schema=schema,
                                        slice_functions=[slice_fn],
                                        infer_type_from_schema=True)

Vous passerez ensuite ces options à la méthode `generate_statistics_from_csv()`. Au moment de l'écriture, la génération de statistiques tranchées ne fonctionne que pour les CSVs, vous devrez donc convertir le dataframe Pandas en CSV. Passer les `slice_stats_options` à la méthode `generate_statistics_from_dataframe()` ne produira pas les résultats attendus.

In [ ]:
# Convert dataframe to CSV since `slice_functions` works only with `tfdv.generate_statistics_from_csv`
CSV_PATH = 'slice_sample.csv'
train_df.to_csv(CSV_PATH)

# Calculate statistics for the sliced dataset
sliced_stats = tfdv.generate_statistics_from_csv(CSV_PATH, stats_options=slice_stats_options)

Avec cela, vous avez maintenant les statistiques pour la tranche de l'ensemble. Celles-ci sont emballées dans un tampon de protocole `DatasetFeatureStatisticsList`. Vous pouvez voir les noms des ensembles de données ci-dessous. Le premier élément de la liste (i.e. index=0) est nommé `All_Examples` et contient juste les statistiques de l'ensemble des données. Les deux éléments suivants (i.e. nommés `sex_Male` et `sex_Female`) sont les jeux de données qui contiennent les statistiques pour les tranches. Il est important de noter que ces jeux de données sont du type : `DatasetFeatureStatistics`. Vous verrez pourquoi c'est important après la cellule ci-dessous.

In [ ]:
print(f'Datasets generated: {[sliced.name for sliced in sliced_stats.datasets]}')

print(f'Type of sliced_stats elements: {type(sliced_stats.datasets[0])}')

Vous pouvez alors visualiser les statistiques comme précédemment pour examiner les tranches. Une mise en garde importante est que `visualize_statistics()` accepte un type `DatasetFeatureStatisticsList` au lieu de `DatasetFeatureStatistics`. Ainsi, au moins pour cette version de TFDV, vous devrez le convertir au type correct.

In [ ]:
from tensorflow_metadata.proto.v0.statistics_pb2 import DatasetFeatureStatisticsList

# Convert `Male` statistics (index=1) to the correct type and get the dataset name
male_stats_list = DatasetFeatureStatisticsList()
male_stats_list.datasets.extend([sliced_stats.datasets[1]])
male_stats_name = sliced_stats.datasets[1].name

# Convert `Female` statistics (index=2) to the correct type and get the dataset name
female_stats_list = DatasetFeatureStatisticsList()
female_stats_list.datasets.extend([sliced_stats.datasets[2]])
female_stats_name = sliced_stats.datasets[2].name

# Visualize the two slices side by side
tfdv.visualize_statistics(
    lhs_statistics=male_stats_list,
    rhs_statistics=female_stats_list,
    lhs_name=male_stats_name,
    rhs_name=female_stats_name
)

Vous devriez maintenant voir la visualisation des deux tranches et vous pouvez comparer comment elles sont représentées dans l'ensemble de données. 

Nous vous encourageons à revenir au début de cette section et à essayer différentes tranches. Voici d'autres possibilités que vous pouvez explorer :

* Si vous voulez être plus spécifique, alors vous pouvez faire correspondre la valeur spécifique au nom de la caractéristique. Par exemple, si vous voulez seulement `Male`, alors vous pouvez le déclarer comme `features={'sex' : [b'Male']}`. Notez que la chaîne littérale doit être passée en octets avec le préfixe `b'`.

* Vous pouvez également passer plusieurs caractéristiques si vous le souhaitez. Par exemple, si vous voulez couper à la fois les caractéristiques `sex` et `race`, vous pouvez faire `features={'sex' : Aucun, 'race' : Aucun}`.

Vous pouvez trouver fastidieux ou inefficace de refaire tout le processus pour une tranche particulière. Pour cela, vous pouvez créer des fonctions d'aide pour rationaliser les conversions de type et vous verrez une implémentation dans le devoir de cette semaine.



* [TFDV Guide](https://www.tensorflow.org/tfx/data_validation/get_started) 
* [TFDV blog post](https://blog.tensorflow.org/2018/09/introducing-tensorflow-data-validation.html)
* [Tensorflow Official Tutorial](https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/data_validation/tfdv_basic.ipynb#scrollTo=mPt5BHTwy_0F)
* [API Docs](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv)